# QuakeMigrate example - Icequake detection at the Rutford Ice Stream, Antarctica

## Overview

This notebook contains an example demonstrating how to run QuakeMigrate for icequake detection.

Here, we detail how to:

1. Calculate a travel-time lookup table for the seismometer network and example grid, using a velocity model with homogeneous P- and S-wave velocities.
2. Run the detect stage, to continuously migrate and stack phase arrival onset functions at each point in the grid, searching for coherent sources of energy in space and time.
3. Run the trigger stage to identify candidate events from the continuous detect output.
4. Run the locate stage to calculate refined locations for these candidate events, and provide a range of additional outputs and plots, including robust location uncertainty estimates and phase picks.

In [ ]:
import pathlib

import pandas as pd
from obspy import UTCDateTime
from obspy.clients.fdsn.mass_downloader import (
    GlobalDomain,
    Restrictions,
    MassDownloader,
)
from obspy.core import AttribDict
from pyproj import Proj

from quakemigrate import QuakeScan, Trigger
from quakemigrate.io import Archive, read_stations
from quakemigrate.lut import compute_traveltimes
from quakemigrate.signal.onsets import STALTAOnset
from quakemigrate.signal.pickers import GaussianPicker

%matplotlib inline

In [ ]:
# --- i/o paths ---
station_file = "./inputs/rutford_stations.txt"
data_in = "./inputs/mSEED"
lut_out = "./outputs/lut/icequake.LUT"
run_path = "./outputs/runs"
run_name = "example_run"

## 0. Download miniSEED data

In [ ]:
data_path = pathlib.Path(data_in)
stationxml_storage = "./inputs/DATALESS"


# --- Define directory structure for storing waveform data ---
def get_mseed_storage(network, station, location, channel, starttime, endtime):
    fname = (
        data_path
        / f"{starttime.year}"
        / f"{starttime.julday:03d}"
        / f"{station}_{channel[2]}.m"
    ).as_posix()

    return fname


# --- Set network code & client ---
network = "YG"
datacentres = ["IRIS"]
# global domain (specifying network and stations instead)
domain = GlobalDomain()

# --- Set time period over which download data ---
starttime = UTCDateTime("2009-01-21T04:00:00.0")
endtime = UTCDateTime("2009-01-21T04:00:20.0")

# --- Read in station file ---
stations = read_stations(station_file)
stations_string = ",".join(stations["Name"])

# --- Set up request ---
restrictions = Restrictions(
    starttime=starttime,
    endtime=endtime,
    chunklength_in_sec=86400,
    network=network,
    station=stations_string,
    channel_priorities=["GL[123]"],
    minimum_interstation_distance_in_m=0,
)

# --- Download waveform data ---
mdl = MassDownloader(providers=datacentres)
mdl.download(
    domain,
    restrictions,
    threads_per_client=3,
    mseed_storage=get_mseed_storage,
    stationxml_storage=stationxml_storage,
)


## 1. Calculate a travel-time lookup table (LUT)

Note: You may find that your system cannot run this example at the intended resolution. If so, you should adjust the node spacing, with the caveat that this will reduce the resolution of the final event location! This can be done either when the lookup table is created (i.e., adjust the line `grid_spec.node_spacing = [0.025, 0.025, 0.025]` to use larger node spacings, e.g., `0.05` or `0.1`) or by decimating this lookup table before the detect/locate stages.

In [ ]:
# --- Read in the station information file ---
stations = read_stations(station_file)

# --- Define the input and grid projections ---
gproj = Proj(
    proj="lcc",
    units="km",
    lon_0=-83.925,
    lat_0=-78.145,
    lat_1=-78.16,
    lat_2=-78.13,
    datum="WGS84",
    ellps="WGS84",
    no_defs=True,
)
cproj = Proj(proj="longlat", datum="WGS84", ellps="WGS84", no_defs=True)

# --- Define the grid specifications ---
# AttribDict behaves like a Python dict, but also has '.'-style access.
grid_spec = AttribDict()
grid_spec.ll_corner = [-84.1, -78.17, 1.0]
grid_spec.ur_corner = [-83.75, -78.12, 3.0]
grid_spec.node_spacing = [0.025, 0.025, 0.025]
grid_spec.grid_proj = gproj
grid_spec.coord_proj = cproj

# --- Homogeneous LUT generation ---
lut = compute_traveltimes(
    grid_spec,
    stations,
    method="homogeneous",
    phases=["P", "S"],
    vp=3.841,
    vs=1.970,
    log=True,
    save_file=lut_out,
)

## 2. Run the detect stage: continuously migrate phase arrival onset functions through the grid to detect coalescence peaks in space and time

In [ ]:
# --- Read in station file ---
stations = read_stations(station_file)

# --- Create new Archive and set path structure ---
archive = Archive(
    archive_path=data_in, stations=stations, archive_format="YEAR/JD/STATION"
)

# --- Create new Onset ---
onset = STALTAOnset(position="classic", sampling_rate=250)
onset.phases = ["P", "S"]
onset.bandpass_filters = {"P": [20, 124, 4], "S": [10, 124, 4]}
onset.sta_lta_windows = {"P": [0.01, 0.25], "S": [0.05, 0.5]}
onset.channel_maps = {"P": "*1", "S": "*[2,3]"}

# --- Decimate the LUT by a factor of 2 in each dimension, if needed ---
# lut = lut.decimate([2, 2, 2,])

# --- Create new QuakeScan ---
scan = QuakeScan(
    archive,
    lut,
    onset=onset,
    run_path=run_path,
    run_name=run_name,
    log=True,
    loglevel="info",
)

# --- Set detect parameters ---
scan.timestep = 1.0
# NOTE: please increase the thread-count as your system allows; the
# core migration routines are compiled against OpenMP, and using
# multithreading will ~ linearly speed up the compute time!
scan.threads = 1

# --- Set time period over which to run detect ---
starttime = "2009-01-21T04:00:05.0"
endtime = "2009-01-21T04:00:10.0"

In [ ]:
# --- Run detect ---
scan.detect(starttime, endtime)

## 3. Run the trigger stage: identify individual icequakes from the continuous detect output

In [ ]:
# --- Create new Trigger ---
trig = Trigger(lut, run_path, run_name, log=True, loglevel="info")

# --- Set trigger parameters ---
trig.marginal_window = 0.06
trig.min_event_interval = 0.12
trig.normalise_coalescence = True

# --- Static threshold ---
trig.threshold_method = "static"
trig.static_threshold = 3.0

# --- Run trigger ---
trig.trigger(starttime, endtime, interactive_plot=False)

### Show the trigger summary PDF file
NOTE: this may not display properly for some OS's / browsers (e.g. Safari on MacOS). If all you see is a grey box, please try opening the notebook in Chrome or Firefox, or open the file directly using a PDF viewer.

In [ ]:
# Show the trigger summary pdf file
icequake_trigger_summary_image_fname = "outputs/runs/icequake_example/trigger/summaries/icequake_example_2009_021_Trigger.pdf"
from IPython.display import IFrame  # For plotting pdf

IFrame(icequake_trigger_summary_image_fname, width=800, height=450)  # Plot pdf

## 4. Run the locate stage: calculate refined locations and location uncertainty estimates

Note: Here we create a new onset object, using the "centred" STALTAOnset. This produces a more accurate gaussian representation of the phase arrival probability density function, and is less phase-shifted. However, it is much more sensitive to sharp offsets due to instrument spikes etc., and is less flexible in identifying arrivals with different frequency content than the "classic" STALTAOnset, so in general that is the better choice to use for detect().

In [ ]:
# --- Create new Onset ---
onset = STALTAOnset(position="centred", sampling_rate=250)
onset.phases = ["P", "S"]
onset.bandpass_filters = {"P": [20, 124, 4], "S": [10, 124, 4]}
onset.sta_lta_windows = {"P": [0.01, 0.25], "S": [0.05, 0.5]}
onset.channel_maps = {"P": "*1", "S": "*[2,3]"}

# --- Create new PhasePicker ---
picker = GaussianPicker(onset=onset)
picker.plot_picks = True

# --- Create new QuakeScan ---
scan = QuakeScan(
    archive,
    lut,
    onset=onset,
    picker=picker,
    run_path=run_path,
    run_name=run_name,
    log=True,
    loglevel="info",
)

# --- Set locate parameters ---
scan.marginal_window = 0.06
# NOTE: please increase the thread-count as your system allows; the
# core migration routines are compiled against OpenMP, and using
# multithreading will ~ linearly speed up the compute time!
scan.threads = 1

# Turn on plotting features
scan.plot_event_summary = True

# --- Toggle writing of waveforms ---
scan.write_cut_waveforms = False

In [ ]:
# --- Run locate ---
scan.locate(starttime=starttime, endtime=endtime)

## 5. Some of the key locate outputs

In [ ]:
# Show the .event file, containing event origin time and location:
icequake_event_fname = (
    "./outputs/runs/example_run/locate/events/20090121040007144.event"
)
event_df = pd.read_csv(icequake_event_fname)

event_df

In [ ]:
# Show the .picks file, containing station time picks:
icequake_pick_fname = (
    "outputs/runs/example_run/locate/picks/20090121040007144.picks"
)
pick_df = pd.read_csv(icequake_pick_fname)

pick_df

In [ ]:
# Show the event summary pdf file, containing event origin time and location, a
# plot displaying the 3D coalescence image, and a waveform gather showing the
# fit of the modelled arrival times to the data:
icequake_event_summary_image_fname = "outputs/runs/example_run/locate/summaries/example_run_20090121040007144_EventSummary.pdf"
from IPython.display import IFrame  # For plotting pdf

IFrame(icequake_event_summary_image_fname, width=800, height=550)  # Plot pdf

# NOTE: this may not display properly for some OS's / browsers (e.g. Safari on
# MacOS). If all you see is a grey box, please try opening the notebook in
# Chrome or Firefox, or open the file directly using a PDF viewer.

In [ ]:
# Show an example of a pick summary pdf file, displaying the onset functions
# for P and S phases, the gaussian fit to the onset function within the pick
# window, and the resulting picktime, uncertainty and SNR.
icequake_event_summary_image_fname = "outputs/runs/example_run/locate/pick_plots/20090121040007144/20090121040007144_ST10.pdf"
from IPython.display import IFrame  # For plotting pdf

IFrame(icequake_event_summary_image_fname, width=800, height=550)  # Plot pdf

# NOTE: this may not display properly for some OS's / browsers (e.g. Safari on
# MacOS). If all you see is a grey box, please try opening the notebook in
# Chrome or Firefox, or open the file directly using a PDF viewer.